In [1]:
import os
import json
import databento as db
import pandas as pd
from datetime import datetime
from pathlib import Path

In [2]:
API_KEY = "Your API Key"
SNAPSHOT_TIME = pd.Timestamp("2025-03-20 21:00:00") # Timezone naive, specified as UTC 
SYMBOLS = ["SR1H5", "SR1J5", "SR1K5", "SR3H5", "SR3M5", "SR3U5", "SR3Z5", "SR3H6", "SR3M6", "SR3U6", "SR3Z6", "SR3H7", "SR3M7", "SR3U7", "SR3Z7",
           "SR3H8", "SR1H5-SR1J5", "SR1J5-SR1K5", "SR3H5-SR3M5", "SR3M5-SR3U5", "SR3U5-SR3Z5", "SR3Z5-SR3H6", "SR3H6-SR3M6", "SR3M6-SR3U6", 
           "SR3U6-SR3Z6","SR3Z6-SR3H7", "SR3H7-SR3M7", "SR3M7-SR3U7", "SR3U7-SR3Z7", "SR3Z7-SR3H8", "SR3:BF H5-M5-U5", "SR3:BF M5-U5-Z5", 
           "SR3:BF U5-Z5-H6","SR3:BF Z5-H6-M6", "SR3:BF H6-M6-U6", "SR3:BF M6-U6-Z6", "SR3:BF U6-Z6-H7", "SR3:BF Z6-H7-M7", "SR3:BF H7-M7-U7", 
           "SR3:BF M7-U7-Z7", "SR3:BF U7-Z7-H8"] 
SAVE_DIR = Path("order_books")
SAVE_DIR.mkdir(exist_ok=True)

client = db.Historical(API_KEY)
all_books = []

for symbol in SYMBOLS:
    print(f"Processing {symbol}...")
    data_path = SAVE_DIR / f"mbp10_{symbol}_20250320_1500CST.dbn.zst" # Rename for date 
    
    if os.path.exists(data_path):
        data = db.DBNStore.from_file(data_path)
    else:
        data = client.timeseries.get_range(
            dataset="GLBX.MDP3",
            start="2025-03-20T15:00:00-06:00", # Specified as CST, returned in UTC 
            end="2025-03-20T15:30:00-06:00",   # Window around tgt snapshot time 
            symbols=[symbol],
            schema="mbp-10",
            path=data_path,
        )

    for record in data:
        ts = pd.to_datetime(record.ts_event, unit="ns")  # Also timezone naive
        if ts >= SNAPSHOT_TIME and record.flags & db.RecordFlags.F_LAST:
            levels = record.levels
            book = {
                "symbol": symbol,
                "snapshot_time": str(ts),
                "bid_prices": [lvl.bid_px * 1e-9 for lvl in levels],
                "bid_sizes": [lvl.bid_sz for lvl in levels],
                "ask_prices": [lvl.ask_px * 1e-9 for lvl in levels],
                "ask_sizes": [lvl.ask_sz for lvl in levels],
            }
            all_books.append(book)
            print(f" Snapshot for {symbol} at {ts} recorded.")
            break

Processing SR1H5...
 Snapshot for SR1H5 at 2025-03-20 21:00:00.082027307 recorded.
Processing SR1J5...
 Snapshot for SR1J5 at 2025-03-20 21:00:00.082027307 recorded.
Processing SR1K5...
 Snapshot for SR1K5 at 2025-03-20 21:00:00.082027307 recorded.
Processing SR3H5...
 Snapshot for SR3H5 at 2025-03-20 21:00:00.082027307 recorded.
Processing SR3M5...
 Snapshot for SR3M5 at 2025-03-20 21:00:00.082027307 recorded.
Processing SR3U5...
 Snapshot for SR3U5 at 2025-03-20 21:00:00.082027307 recorded.
Processing SR3Z5...
 Snapshot for SR3Z5 at 2025-03-20 21:00:00.082027307 recorded.
Processing SR3H6...
 Snapshot for SR3H6 at 2025-03-20 21:00:00.082027307 recorded.
Processing SR3M6...
 Snapshot for SR3M6 at 2025-03-20 21:00:00.082027307 recorded.
Processing SR3U6...
 Snapshot for SR3U6 at 2025-03-20 21:00:00.082027307 recorded.
Processing SR3Z6...
 Snapshot for SR3Z6 at 2025-03-20 21:00:00.082027307 recorded.
Processing SR3H7...
 Snapshot for SR3H7 at 2025-03-20 21:00:00.082027307 recorded.
Proc

In [3]:
# Save as JSON 
json_path = SAVE_DIR / "all_order_books_0320.json"
with open(json_path, "w") as f:
    json.dump(all_books, f, indent=2)

print(f"\n All books saved to: {json_path}")


 All books saved to: order_books\all_order_books_0320.json
